In [26]:

def carregar_sequencia(filename):
    from Bio import SeqIO
    
    # Lemos o ficheiro .fasta
    seq_proteina = SeqIO.read(open(filename),format='fasta')

    sp = seq_proteina.seq # Isolamos a sequência

    return sp

In [27]:
# Procedemos à contagem de aminoácidos:
# Primeiro para a sequência traduzida e depois para o conjunto de sequências após a remoção de codões stop

sp = carregar_sequencia('output\\UQL50880.fasta')

def conta_aa(seq_prot):
    # TODO colocar um dicionário bonitinho
    from collections import Counter
    contagem_aa = dict(Counter(seq_prot))
    
    return contagem_aa

# conta_aa(sp)


In [29]:
swiss_id = 'P01906' # Encontra-se o ID no ficheiro FASTA 

def swiss_prot_scan(swiss_id):
    from Bio import SeqIO
    from Bio import ExPASy
    # Efetuamos uma pesquisa na base de dados SwissProt 
    handle = ExPASy.get_sprot_raw(swiss_id) # Encontramos o ID ao ler o ficheiro fasta
    sr = SeqIO.read(handle, "swiss")
    print(
        f'ID {sr.id}',
        f'Sequência: {sr.seq}',
        f'Tamanho da sequência: {len(sr.seq)} bp',
        f'Nome: {sr.name}',
        f'Descrição: {sr.description}',
        f'Taxonomia: {sr.annotations["taxonomy"]}',
        f'Organismo: {sr.annotations["organism"]}',
        f'Keywords: {sr.annotations["keywords"]}',
        sep = '\n')

swiss_prot_scan('P01906')

ID P01906
Sequência: MILNKALLLGALALTAVMSPCGGEDIVADHVASYGVNFYQSHGPSGQYTHEFDGDEEFYVDLETKETVWQLPMFSKFISFDPQSALRNMAVGKHTLEFMMRQSNSTAATNEVPEVTVFSKFPVTLGQPNTLICLVDNIFPPVVNITWLSNGHSVTEGVSETSFLSKSDHSFFKISYLTFLPSADEIYDCKVEHWGLDEPLLKHWEPEIPAPMSELTETLVCALGLSVGLMGIVVGTVFIIQGLRSVGASRHQGLL
Tamanho da sequência: 255 bp
Nome: DQA2_HUMAN
Descrição: RecName: Full=HLA class II histocompatibility antigen, DQ alpha 2 chain; AltName: Full=DX alpha chain; AltName: Full=HLA class II histocompatibility antigen, DQ(6) alpha chain; AltName: Full=HLA-DQA1; AltName: Full=MHC class II DQA2; Flags: Precursor;
Taxonomia: ['Eukaryota', 'Metazoa', 'Chordata', 'Craniata', 'Vertebrata', 'Euteleostomi', 'Mammalia', 'Eutheria', 'Euarchontoglires', 'Primates', 'Haplorrhini', 'Catarrhini', 'Hominidae', 'Homo']
Organismo: Homo sapiens (Human)
Keywords: ['Adaptive immunity', 'Cell membrane', 'Disulfide bond', 'Endoplasmic reticulum', 'Endosome', 'Glycoprotein', 'Golgi apparatus', 'Immunity', 'Lysosome', 'Membrane', 'MHC II', '

In [30]:
def analise_blast(email, sequencia_proteina, filename, lim = None):
    import os
    from Bio.Blast import NCBIWWW
    
    NCBIWWW.email = email

    
    
    # Verifica se já existe um ficheiro XML para os resultados do BLAST. Ignora a pesquisa caso já exista.
    # Para poupar tempo.
    # Se o ficheiro já existir executa o parsing do ficheiro.

    if '.xml' not in filename:
        filename = filename + '.xml'

    if os.path.isfile(filename): 
        
        blast_parse(filename,lim)

    else: 
    # Executa o BLAST da Proteína
        handle = NCBIWWW.qblast('blastp','swissprot',sequencia_proteina.seq)

        with open(filename, 'w') as _:
            _.write(handle.read())
        
        handle.close()

# analise_blast('pg21019@alunos.uminho.pt', seq_proteina,'blast_results.xml')

In [31]:
filename = 'blast_results.xml'

def blast_parse(filename, lim = None):
    
    from Bio.Blast import NCBIXML

    handle = open(filename)
    blast_res = NCBIXML.parse(handle)
    blast_records = list(blast_res)

    num = 1
    for record in blast_records:

        if lim == None: lim = len(record.alignments)    

        for aligment in record.alignments[0:lim]:
            #print(aligment)
            
            for hsp in aligment.hsps:
                #print(hsp)
                
                if hsp.expect < 0.05:
                    print('****Alinhamento {} ****'.format(num))
                    print('Sequência:',aligment.title)
                    print('Tamanho da Sequência:',aligment.length)
                    print('e-value:',hsp.expect)
                    print(hsp.query[0:40]+'...')
                    print(hsp.match[0:40]+'...')
                    print(hsp.sbjct[0:40]+'...')
                    print()
                    num += 1    
